# Data Input

In [6]:
import pandas as pd
import numpy as np
from python_speech_features import mfcc
from python_speech_features import logfbank
import os
import scipy.io.wavfile as wav
from sklearn.decomposition import PCA

actors = [f for f in os.listdir("./voice") if os.path.isdir(os.path.join("./voice",f))]
minase = [ f for f in os.listdir("./voice/minase") ]
matsuoka = [ f for f in os.listdir("./voice/matsuoka") ]
takahashi = [ f for f in os.listdir("./voice/takahashi") ]
kobayashi = [ f for f in os.listdir("./voice/kobayashi") ]

df = pd.DataFrame(columns=["role","actor"])
for i in minase:
    df = df.append({'role':i,'actor':"minase"},ignore_index=True)
for i in matsuoka:
    df = df.append({'role':i,'actor':"matsuoka"},ignore_index=True)
for i in takahashi:
    df = df.append({'role':i,'actor':"takahashi"},ignore_index=True)
for i in kobayashi:
    df = df.append({'role':i,'actor':"kobayashi"},ignore_index=True)
    
x = df['role']
y = df['actor']

nn_x = []
trad_x = []
# rate,data = wav.read("./voice/minase/hestia_1.wav")
# t = mfcc(signal=data,samplerate=rate,winlen=0.5,winstep=0.1,nfft=22050,numcep=1)

for i,j in enumerate(x):
    #print(y.iloc[i])
    rate,data = wav.read("./voice/"+y.iloc[i]+"/"+j)
    t = mfcc(signal=data,samplerate=rate,winlen=0.5,winstep=data.shape[0]/rate/5,nfft=data.shape[0],numcep=13)
    t1 = mfcc(signal=data,samplerate=rate,winlen=0.5,winstep=data.shape[0]/rate/5,nfft=data.shape[0],numcep=1)
#     pca = PCA(n_components=1)
#     g = pca.fit_transform(t)
#     g = np.append([],g)
#     trans_pca.append(g)
#     print(t.shape,j)
    t1=np.append([],t1)
    nn_x.append(t)
    trad_x.append(t1)
nn_x=np.asarray(nn_x)
trad_x=np.asarray(trad_x)
print("done!")

done!


In [ ]:
trans_x

# Data Preprocessing

In [7]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

oh = OneHotEncoder(sparse=False)
le = LabelEncoder()

to_train = le.fit_transform(y)


x_train, x_test, y_train ,y_test = train_test_split(trad_x,y,test_size=0.3,random_state=8)
## ann use
x_train, x_test, y_train1 ,y_test1 = train_test_split(trad_x,to_train,test_size=0.3,random_state=8)
## cnn use
cx_train, cx_test, cy_train1 ,cy_test1 = train_test_split(nn_x,to_train,test_size=0.3,random_state=8)

y_train2 = oh.fit_transform(np.array(y_train1).reshape(-1,1))
y_test2 = oh.fit_transform(np.array(y_test1).reshape(-1,1))

cy_train2 = oh.fit_transform(np.array(cy_train1).reshape(-1,1))
cy_test2 = oh.fit_transform(np.array(cy_test1).reshape(-1,1))

# px_train, px_test, py_train ,py_test = train_test_split(trans_pca,y,test_size=0.3,random_state=8)

# Model Construction

In [8]:
from sklearn.metrics import accuracy_score,confusion_matrix,recall_score,precision_score
from sklearn.cluster import KMeans
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC 
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import GaussianNB as gb

def scoretest(clf,px_test,y_test):

    pred = clf.predict(px_test)
    acc = accuracy_score(y_test,pred)
    cm = confusion_matrix(y_test,pred)
    rec = recall_score(y_test,pred,average='macro')
    prec = precision_score(y_test,pred,average='macro')
    return acc,cm,rec,prec

def svm(x_train,y_train,x_test,y_test):

    parameters = {'kernel':('linear','poly','rbf'),'C':[0.1,1, 10, 100],'gamma':[1,0.1,0.01,0.001]}

    #print(kernel)
    svc =SVC(kernel='rbf',C=1,gamma=0.01)
    svc.fit(x_train,y_train)
    acc,cm,rec,prec = scoretest(clf,x_test,y_test)
    return acc,cm,rec,prec

def gaussian(x_train,y_train,x_test,y_test):
    clf = gb()
    
    clf.fit(x_train,y_train)

    print(clf.predict([x_test[0]]),y_test.iloc[0])
    acc,cm,rec,prec = scoretest(clf,x_test,y_test)
    return acc,cm,rec,prec



In [9]:
from sklearn.svm import SVC 
from sklearn.model_selection import GridSearchCV

parameters = {'kernel':('linear','poly','rbf'),'C':[0.1,1, 10, 100],'gamma':[1,0.1,0.01,0.001]}


svc=SVC()
clf = GridSearchCV(svc,parameters,verbose=3)
clf.fit(x_train,y_train)

Fitting 5 folds for each of 48 candidates, totalling 240 fits
[CV 1/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.263 total time=   0.0s
[CV 2/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.263 total time=   0.0s
[CV 3/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.263 total time=   0.0s
[CV 4/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.389 total time=   0.0s
[CV 5/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.278 total time=   0.0s
[CV 1/5] END .......C=0.1, gamma=1, kernel=poly;, score=0.474 total time=   0.0s
[CV 2/5] END .......C=0.1, gamma=1, kernel=poly;, score=0.316 total time=   0.0s
[CV 3/5] END .......C=0.1, gamma=1, kernel=poly;, score=0.316 total time=   0.0s
[CV 4/5] END .......C=0.1, gamma=1, kernel=poly;, score=0.333 total time=   0.0s
[CV 5/5] END .......C=0.1, gamma=1, kernel=poly;, score=0.222 total time=   0.0s
[CV 1/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.316 total time=   0.0s
[CV 2/5] END ........C=0.1, gamma=1, kernel=rbf

[CV 5/5] END ...C=1, gamma=0.001, kernel=linear;, score=0.333 total time=   0.0s
[CV 1/5] END .....C=1, gamma=0.001, kernel=poly;, score=0.316 total time=   0.0s
[CV 2/5] END .....C=1, gamma=0.001, kernel=poly;, score=0.263 total time=   0.0s
[CV 3/5] END .....C=1, gamma=0.001, kernel=poly;, score=0.316 total time=   0.0s
[CV 4/5] END .....C=1, gamma=0.001, kernel=poly;, score=0.278 total time=   0.0s
[CV 5/5] END .....C=1, gamma=0.001, kernel=poly;, score=0.389 total time=   0.0s
[CV 1/5] END ......C=1, gamma=0.001, kernel=rbf;, score=0.421 total time=   0.0s
[CV 2/5] END ......C=1, gamma=0.001, kernel=rbf;, score=0.316 total time=   0.0s
[CV 3/5] END ......C=1, gamma=0.001, kernel=rbf;, score=0.263 total time=   0.0s
[CV 4/5] END ......C=1, gamma=0.001, kernel=rbf;, score=0.333 total time=   0.0s
[CV 5/5] END ......C=1, gamma=0.001, kernel=rbf;, score=0.389 total time=   0.0s
[CV 1/5] END ......C=10, gamma=1, kernel=linear;, score=0.421 total time=   0.2s
[CV 2/5] END ......C=10, gam

[CV 2/5] END ..C=100, gamma=0.01, kernel=linear;, score=0.263 total time=   0.9s
[CV 3/5] END ..C=100, gamma=0.01, kernel=linear;, score=0.316 total time=   1.8s
[CV 4/5] END ..C=100, gamma=0.01, kernel=linear;, score=0.389 total time=   6.5s
[CV 5/5] END ..C=100, gamma=0.01, kernel=linear;, score=0.389 total time=   1.0s
[CV 1/5] END ....C=100, gamma=0.01, kernel=poly;, score=0.474 total time=   0.0s
[CV 2/5] END ....C=100, gamma=0.01, kernel=poly;, score=0.316 total time=   0.0s
[CV 3/5] END ....C=100, gamma=0.01, kernel=poly;, score=0.316 total time=   0.0s
[CV 4/5] END ....C=100, gamma=0.01, kernel=poly;, score=0.333 total time=   0.0s
[CV 5/5] END ....C=100, gamma=0.01, kernel=poly;, score=0.222 total time=   0.0s
[CV 1/5] END .....C=100, gamma=0.01, kernel=rbf;, score=0.474 total time=   0.0s
[CV 2/5] END .....C=100, gamma=0.01, kernel=rbf;, score=0.421 total time=   0.0s
[CV 3/5] END .....C=100, gamma=0.01, kernel=rbf;, score=0.368 total time=   0.0s
[CV 4/5] END .....C=100, gam

GridSearchCV(estimator=SVC(),
             param_grid={'C': [0.1, 1, 10, 100], 'gamma': [1, 0.1, 0.01, 0.001],
                         'kernel': ('linear', 'poly', 'rbf')},
             verbose=3)

In [10]:
res = pd.DataFrame(clf.cv_results_)
res = res[['param_C','param_gamma','param_kernel','mean_test_score','rank_test_score']]
res = res.sort_values(by=['rank_test_score'])
res = res.reset_index(drop=True)
res.groupby('param_kernel').first()

,param_C,param_gamma,mean_test_score,rank_test_score
param_kernel,,,,
linear,100,0.010,0.355556,5
poly,10,0.001,0.342105,21
rbf,10,0.010,0.408187,1


# Neural Network

In [11]:
## mlp

import torch
from torch import nn
import torch.nn.functional as F

class NeuralNetwork(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNetwork, self).__init__()
        self.layer1 = nn.Sequential(  
            nn.Linear(input_size,hidden_size[0]),
            nn.ReLU()
        )
        self.layer2 = nn.Sequential(
            nn.Linear(hidden_size[0],hidden_size[1]),
            nn.ReLU(),
        )
        self.layer3 = nn.Sequential(
            nn.Linear(hidden_size[1],hidden_size[2]),
            nn.ReLU(),
        )
        self.fc = nn.Linear(hidden_size[2],num_classes)
        
    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = x.reshape(x.size(0),-1)
        x = self.fc(x)
        x = F.softmax(x,dim=1)
        return x
    
def mlp(x_test):
    pred = []
    name = []
    with torch.no_grad():
        for index,listing in enumerate(x_test):
            lists = torch.FloatTensor(listing).to(device)
            lists = lists.view(-1,x_test.shape[1])
            outputs = model(lists)
           # outputs = torch.Tensor.cpu(outputs).numpy()
            pred.append(torch.Tensor.cpu(torch.argmax(outputs)).numpy().item())
#             print(pred[-1])
#             name.append(le.inverse_transform([torch.Tensor.cpu(torch.argmax(outputs)).numpy().item()]))
#             print(name[-1])
            
        acc = accuracy_score(y_test1,pred)
        cm = confusion_matrix(y_test1,pred)
        rec = recall_score(y_test1,pred,average='macro')
        prec = precision_score(y_test1,pred,average='macro')
        return acc,cm,rec,prec

In [12]:
# Define Hyper-parameters 
input_size = x_train.shape[1]
hidden_size = [24,16,8,12,8]
num_classes = 4
num_epochs = 100
learning_rate = 0.001

device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = NeuralNetwork(input_size, hidden_size, num_classes).to(device)
print(model)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)  
total_step = len(x_train)

for epoch in range(num_epochs):
    for index,listing in enumerate(x_train):
        # Forward pass
        lists = torch.FloatTensor(listing).to(device)
        lists = lists.view(-1,x_train.shape[1])
#         print(lists)
        outputs = model(lists)
        
        label = torch.FloatTensor([y_train2[index]]).to(device)
        label = label.view(-1,4)
#         outputs = outputs.view(1,-1)
#         print(outputs,label)
        
        loss = criterion(outputs, label)

        # Backprpagation and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

#         if (index+1) % 1000 == 0:
        print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
               .format(epoch+1, num_epochs, index+1, total_step, loss.item()))


NeuralNetwork(
  (layer1): Sequential(
    (0): Linear(in_features=11, out_features=24, bias=True)
    (1): ReLU()
  )
  (layer2): Sequential(
    (0): Linear(in_features=24, out_features=16, bias=True)
    (1): ReLU()
  )
  (layer3): Sequential(
    (0): Linear(in_features=16, out_features=8, bias=True)
    (1): ReLU()
  )
  (fc): Linear(in_features=8, out_features=4, bias=True)
)


C:\Users\kienn\AppData\Local\Temp/ipykernel_20204/1272109559.py:23: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ..\torch\csrc\utils\tensor_new.cpp:201.)
  label = torch.FloatTensor([y_train2[index]]).to(device)


Epoch [1/100], Step [1/93], Loss: 1.3343
Epoch [1/100], Step [2/93], Loss: 1.4799
Epoch [1/100], Step [3/93], Loss: 1.4111
Epoch [1/100], Step [4/93], Loss: 1.3975
Epoch [1/100], Step [5/93], Loss: 1.2954
Epoch [1/100], Step [6/93], Loss: 1.4036
Epoch [1/100], Step [7/93], Loss: 1.3171
Epoch [1/100], Step [8/93], Loss: 1.2723
Epoch [1/100], Step [9/93], Loss: 1.4027
Epoch [1/100], Step [10/93], Loss: 1.2776
Epoch [1/100], Step [11/93], Loss: 1.4705
Epoch [1/100], Step [12/93], Loss: 1.3638
Epoch [1/100], Step [13/93], Loss: 1.5414
Epoch [1/100], Step [14/93], Loss: 1.5174
Epoch [1/100], Step [15/93], Loss: 1.5190
Epoch [1/100], Step [16/93], Loss: 1.4002
Epoch [1/100], Step [17/93], Loss: 1.3906
Epoch [1/100], Step [18/93], Loss: 1.3982
Epoch [1/100], Step [19/93], Loss: 1.2933
Epoch [1/100], Step [20/93], Loss: 1.5299
Epoch [1/100], Step [21/93], Loss: 1.3930
Epoch [1/100], Step [22/93], Loss: 1.3846
Epoch [1/100], Step [23/93], Loss: 1.2793
Epoch [1/100], Step [24/93], Loss: 1.3987
E

Epoch [3/100], Step [39/93], Loss: 1.5090
Epoch [3/100], Step [40/93], Loss: 1.2771
Epoch [3/100], Step [41/93], Loss: 1.3532
Epoch [3/100], Step [42/93], Loss: 1.2850
Epoch [3/100], Step [43/93], Loss: 1.4590
Epoch [3/100], Step [44/93], Loss: 1.4094
Epoch [3/100], Step [45/93], Loss: 1.5838
Epoch [3/100], Step [46/93], Loss: 1.2878
Epoch [3/100], Step [47/93], Loss: 1.3281
Epoch [3/100], Step [48/93], Loss: 1.2805
Epoch [3/100], Step [49/93], Loss: 1.4526
Epoch [3/100], Step [50/93], Loss: 1.2845
Epoch [3/100], Step [51/93], Loss: 1.4864
Epoch [3/100], Step [52/93], Loss: 1.2978
Epoch [3/100], Step [53/93], Loss: 1.5004
Epoch [3/100], Step [54/93], Loss: 1.3700
Epoch [3/100], Step [55/93], Loss: 1.3088
Epoch [3/100], Step [56/93], Loss: 1.4879
Epoch [3/100], Step [57/93], Loss: 1.2677
Epoch [3/100], Step [58/93], Loss: 1.4726
Epoch [3/100], Step [59/93], Loss: 1.5464
Epoch [3/100], Step [60/93], Loss: 1.2957
Epoch [3/100], Step [61/93], Loss: 1.4646
Epoch [3/100], Step [62/93], Loss:

Epoch [6/100], Step [3/93], Loss: 1.4280
Epoch [6/100], Step [4/93], Loss: 0.8682
Epoch [6/100], Step [5/93], Loss: 1.2525
Epoch [6/100], Step [6/93], Loss: 0.9002
Epoch [6/100], Step [7/93], Loss: 1.2577
Epoch [6/100], Step [8/93], Loss: 1.2794
Epoch [6/100], Step [9/93], Loss: 1.3613
Epoch [6/100], Step [10/93], Loss: 1.2375
Epoch [6/100], Step [11/93], Loss: 1.4750
Epoch [6/100], Step [12/93], Loss: 1.2972
Epoch [6/100], Step [13/93], Loss: 1.6915
Epoch [6/100], Step [14/93], Loss: 1.4514
Epoch [6/100], Step [15/93], Loss: 1.4437
Epoch [6/100], Step [16/93], Loss: 1.3377
Epoch [6/100], Step [17/93], Loss: 1.3754
Epoch [6/100], Step [18/93], Loss: 1.3788
Epoch [6/100], Step [19/93], Loss: 1.3300
Epoch [6/100], Step [20/93], Loss: 1.4452
Epoch [6/100], Step [21/93], Loss: 1.4301
Epoch [6/100], Step [22/93], Loss: 1.3635
Epoch [6/100], Step [23/93], Loss: 1.2799
Epoch [6/100], Step [24/93], Loss: 1.3741
Epoch [6/100], Step [25/93], Loss: 1.4249
Epoch [6/100], Step [26/93], Loss: 1.3198

...

Epoch [90/100], Step [66/93], Loss: 0.7488
Epoch [90/100], Step [67/93], Loss: 0.7886
Epoch [90/100], Step [68/93], Loss: 1.3067
Epoch [90/100], Step [69/93], Loss: 0.7701
Epoch [90/100], Step [70/93], Loss: 1.7430
Epoch [90/100], Step [71/93], Loss: 1.7430
Epoch [90/100], Step [72/93], Loss: 0.7572
Epoch [90/100], Step [73/93], Loss: 1.6779
Epoch [90/100], Step [74/93], Loss: 0.7453
Epoch [90/100], Step [75/93], Loss: 1.7435
Epoch [90/100], Step [76/93], Loss: 1.7436
Epoch [90/100], Step [77/93], Loss: 1.7435
Epoch [90/100], Step [78/93], Loss: 0.7491
Epoch [90/100], Step [79/93], Loss: 1.7406
Epoch [90/100], Step [80/93], Loss: 0.7461
Epoch [90/100], Step [81/93], Loss: 0.7738
Epoch [90/100], Step [82/93], Loss: 1.7425
Epoch [90/100], Step [83/93], Loss: 0.7681
Epoch [90/100], Step [84/93], Loss: 0.7454
Epoch [90/100], Step [85/93], Loss: 0.7437
Epoch [90/100], Step [86/93], Loss: 0.7439
Epoch [90/100], Step [87/93], Loss: 0.7472
Epoch [90/100], Step [88/93], Loss: 0.7456
Epoch [90/1

Epoch [92/100], Step [87/93], Loss: 0.7446
Epoch [92/100], Step [88/93], Loss: 0.7492
Epoch [92/100], Step [89/93], Loss: 0.7437
Epoch [92/100], Step [90/93], Loss: 0.7595
Epoch [92/100], Step [91/93], Loss: 0.7598
Epoch [92/100], Step [92/93], Loss: 0.7449
Epoch [92/100], Step [93/93], Loss: 0.7449
Epoch [93/100], Step [1/93], Loss: 1.6743
Epoch [93/100], Step [2/93], Loss: 1.7356
Epoch [93/100], Step [3/93], Loss: 0.8210
Epoch [93/100], Step [4/93], Loss: 0.7437
Epoch [93/100], Step [5/93], Loss: 0.8182
Epoch [93/100], Step [6/93], Loss: 0.7482
Epoch [93/100], Step [7/93], Loss: 1.7389
Epoch [93/100], Step [8/93], Loss: 0.8163
Epoch [93/100], Step [9/93], Loss: 0.7440
Epoch [93/100], Step [10/93], Loss: 0.7446
Epoch [93/100], Step [11/93], Loss: 1.7347
Epoch [93/100], Step [12/93], Loss: 0.7482
Epoch [93/100], Step [13/93], Loss: 1.7437
Epoch [93/100], Step [14/93], Loss: 0.7438
Epoch [93/100], Step [15/93], Loss: 0.7437
Epoch [93/100], Step [16/93], Loss: 0.7626
Epoch [93/100], Step

Epoch [95/100], Step [17/93], Loss: 0.7517
Epoch [95/100], Step [18/93], Loss: 0.7584
Epoch [95/100], Step [19/93], Loss: 1.1175
Epoch [95/100], Step [20/93], Loss: 0.7437
Epoch [95/100], Step [21/93], Loss: 1.7422
Epoch [95/100], Step [22/93], Loss: 0.7731
Epoch [95/100], Step [23/93], Loss: 1.6179
Epoch [95/100], Step [24/93], Loss: 0.7501
Epoch [95/100], Step [25/93], Loss: 1.7430
Epoch [95/100], Step [26/93], Loss: 0.7437
Epoch [95/100], Step [27/93], Loss: 1.6694
Epoch [95/100], Step [28/93], Loss: 0.7494
Epoch [95/100], Step [29/93], Loss: 0.7439
Epoch [95/100], Step [30/93], Loss: 0.7437
Epoch [95/100], Step [31/93], Loss: 0.7554
Epoch [95/100], Step [32/93], Loss: 0.7468
Epoch [95/100], Step [33/93], Loss: 0.7437
Epoch [95/100], Step [34/93], Loss: 0.7476
Epoch [95/100], Step [35/93], Loss: 0.7572
Epoch [95/100], Step [36/93], Loss: 1.7405
Epoch [95/100], Step [37/93], Loss: 0.7571
Epoch [95/100], Step [38/93], Loss: 0.7437
Epoch [95/100], Step [39/93], Loss: 1.6952
Epoch [95/1

Epoch [97/100], Step [40/93], Loss: 0.7514
Epoch [97/100], Step [41/93], Loss: 1.6800
Epoch [97/100], Step [42/93], Loss: 0.7439
Epoch [97/100], Step [43/93], Loss: 0.7514
Epoch [97/100], Step [44/93], Loss: 1.7437
Epoch [97/100], Step [45/93], Loss: 0.7567
Epoch [97/100], Step [46/93], Loss: 0.7524
Epoch [97/100], Step [47/93], Loss: 1.7436
Epoch [97/100], Step [48/93], Loss: 0.7440
Epoch [97/100], Step [49/93], Loss: 0.7440
Epoch [97/100], Step [50/93], Loss: 0.7451
Epoch [97/100], Step [51/93], Loss: 1.6843
Epoch [97/100], Step [52/93], Loss: 0.7437
Epoch [97/100], Step [53/93], Loss: 0.7530
Epoch [97/100], Step [54/93], Loss: 1.0017
Epoch [97/100], Step [55/93], Loss: 1.7429
Epoch [97/100], Step [56/93], Loss: 1.6156
Epoch [97/100], Step [57/93], Loss: 0.7918
Epoch [97/100], Step [58/93], Loss: 0.7558
Epoch [97/100], Step [59/93], Loss: 1.7292
Epoch [97/100], Step [60/93], Loss: 0.7456
Epoch [97/100], Step [61/93], Loss: 0.7533
Epoch [97/100], Step [62/93], Loss: 0.7453
Epoch [97/1

Epoch [99/100], Step [45/93], Loss: 0.7533
Epoch [99/100], Step [46/93], Loss: 0.7554
Epoch [99/100], Step [47/93], Loss: 1.7436
Epoch [99/100], Step [48/93], Loss: 0.7444
Epoch [99/100], Step [49/93], Loss: 0.7439
Epoch [99/100], Step [50/93], Loss: 0.7455
Epoch [99/100], Step [51/93], Loss: 1.6673
Epoch [99/100], Step [52/93], Loss: 0.7437
Epoch [99/100], Step [53/93], Loss: 0.7539
Epoch [99/100], Step [54/93], Loss: 0.7735
Epoch [99/100], Step [55/93], Loss: 1.7405
Epoch [99/100], Step [56/93], Loss: 1.6910
Epoch [99/100], Step [57/93], Loss: 0.7459
Epoch [99/100], Step [58/93], Loss: 0.8007
Epoch [99/100], Step [59/93], Loss: 1.7178
Epoch [99/100], Step [60/93], Loss: 0.7480
Epoch [99/100], Step [61/93], Loss: 0.7465
Epoch [99/100], Step [62/93], Loss: 0.7556
Epoch [99/100], Step [63/93], Loss: 0.7450
Epoch [99/100], Step [64/93], Loss: 1.1212
Epoch [99/100], Step [65/93], Loss: 0.7438
Epoch [99/100], Step [66/93], Loss: 0.7489
Epoch [99/100], Step [67/93], Loss: 0.7544
Epoch [99/1

In [25]:
import torch
from torch import nn
import torch.nn.functional as F

class convnn(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(convnn, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1,6,3),
            nn.ReLU(),
            nn.MaxPool2d(2,2)
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(6,16,3),
            nn.ReLU(),
            nn.MaxPool2d(2,2)
        )
        self.layer3 = nn.Sequential(
            nn.Linear(16 ,20),
            nn.ReLU(),
            nn.Linear(20,12),
            nn.ReLU(),
            nn.Linear(12,4),
        )
        
    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = torch.flatten(x,1)
        x = self.layer3(x)
#         x = F.softmax(x,dim=1)
        return x


def cnn(x_test):
    pred = []
    name = []
    with torch.no_grad():
        for index,listing in enumerate(x_test):
            lists = torch.FloatTensor(listing).to(device)
            lists = lists.view(1,1,11,13)
            outputs = cnn_model(lists)
           # outputs = torch.Tensor.cpu(outputs).numpy()
            pred.append(torch.Tensor.cpu(torch.argmax(outputs)).numpy().item())
#             print(pred[-1])
#             name.append(le.inverse_transform([torch.Tensor.cpu(torch.argmax(outputs)).numpy().item()]))
#             print(name[-1])
            
        acc = accuracy_score(y_test1,pred)
        cm = confusion_matrix(y_test1,pred)
        rec = recall_score(y_test1,pred,average='macro')
        prec = precision_score(y_test1,pred,average='macro')
        return acc,cm,rec,prec

In [26]:
# Define Hyper-parameters 

num_classes = 4
num_epochs = 100
learning_rate = 0.001

device = 'cuda' if torch.cuda.is_available() else 'cpu'
cnn_model = convnn(input_size, hidden_size, num_classes).to(device)
print(cnn_model)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(cnn_model.parameters(), lr=learning_rate)  
total_step = len(x_train)

for epoch in range(num_epochs):
    for index,listing in enumerate(cx_train):
        # Forward pass
        lists = torch.FloatTensor(listing).to(device)
        lists = lists.view(1,1,11,13)
#         print(lists)
        outputs = cnn_model(lists)
        
        label = torch.FloatTensor([cy_train2[index]]).to(device)
        label = label.view(-1,4)
#         outputs = outputs.view(1,-1)
#         print(outputs,label)
        
        loss = criterion(outputs, label)

        # Backprpagation and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

#         if (index+1) % 1000 == 0:
        print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
               .format(epoch+1, num_epochs, index+1, total_step, loss.item()))


convnn(
  (layer1): Sequential(
    (0): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer3): Sequential(
    (0): Linear(in_features=16, out_features=20, bias=True)
    (1): ReLU()
    (2): Linear(in_features=20, out_features=12, bias=True)
    (3): ReLU()
    (4): Linear(in_features=12, out_features=4, bias=True)
  )
)
Epoch [1/100], Step [1/93], Loss: 1.1781
Epoch [1/100], Step [2/93], Loss: 1.4458
Epoch [1/100], Step [3/93], Loss: 2.1103
Epoch [1/100], Step [4/93], Loss: 1.2568
Epoch [1/100], Step [5/93], Loss: 0.9873
Epoch [1/100], Step [6/93], Loss: 1.1665
Epoch [1/100], Step [7/93], Loss: 1.2683
Epoch [1/100], Step [8/93], Loss: 0.9921
Epoch [1/100], Step [9/93], Loss: 1.9295
Epoch [1/10

Epoch [3/100], Step [24/93], Loss: 1.4133
Epoch [3/100], Step [25/93], Loss: 1.4860
Epoch [3/100], Step [26/93], Loss: 1.4336
Epoch [3/100], Step [27/93], Loss: 1.4366
Epoch [3/100], Step [28/93], Loss: 1.3054
Epoch [3/100], Step [29/93], Loss: 1.2617
Epoch [3/100], Step [30/93], Loss: 1.3191
Epoch [3/100], Step [31/93], Loss: 1.3073
Epoch [3/100], Step [32/93], Loss: 1.4606
Epoch [3/100], Step [33/93], Loss: 1.2593
Epoch [3/100], Step [34/93], Loss: 1.3188
Epoch [3/100], Step [35/93], Loss: 1.5316
Epoch [3/100], Step [36/93], Loss: 1.4980
Epoch [3/100], Step [37/93], Loss: 1.2779
Epoch [3/100], Step [38/93], Loss: 1.4963
Epoch [3/100], Step [39/93], Loss: 1.2591
Epoch [3/100], Step [40/93], Loss: 1.3263
Epoch [3/100], Step [41/93], Loss: 1.4961
Epoch [3/100], Step [42/93], Loss: 1.3269
Epoch [3/100], Step [43/93], Loss: 1.2583
Epoch [3/100], Step [44/93], Loss: 1.3271
Epoch [3/100], Step [45/93], Loss: 1.2579
Epoch [3/100], Step [46/93], Loss: 1.3271
Epoch [3/100], Step [47/93], Loss:

Epoch [5/100], Step [41/93], Loss: 1.4046
Epoch [5/100], Step [42/93], Loss: 1.4109
Epoch [5/100], Step [43/93], Loss: 1.2447
Epoch [5/100], Step [44/93], Loss: 1.5099
Epoch [5/100], Step [45/93], Loss: 1.2463
Epoch [5/100], Step [46/93], Loss: 1.4227
Epoch [5/100], Step [47/93], Loss: 1.0583
Epoch [5/100], Step [48/93], Loss: 1.3880
Epoch [5/100], Step [49/93], Loss: 1.2667
Epoch [5/100], Step [50/93], Loss: 1.4261
Epoch [5/100], Step [51/93], Loss: 1.2482
Epoch [5/100], Step [52/93], Loss: 0.9523
Epoch [5/100], Step [53/93], Loss: 1.2813
Epoch [5/100], Step [54/93], Loss: 1.2685
Epoch [5/100], Step [55/93], Loss: 1.2496
Epoch [5/100], Step [56/93], Loss: 1.6444
Epoch [5/100], Step [57/93], Loss: 1.1458
Epoch [5/100], Step [58/93], Loss: 1.9309
Epoch [5/100], Step [59/93], Loss: 1.3011
Epoch [5/100], Step [60/93], Loss: 1.3527
Epoch [5/100], Step [61/93], Loss: 1.2884
Epoch [5/100], Step [62/93], Loss: 1.7671
Epoch [5/100], Step [63/93], Loss: 0.9831
Epoch [5/100], Step [64/93], Loss:

Epoch [7/100], Step [59/93], Loss: 1.1217
Epoch [7/100], Step [60/93], Loss: 1.4897
Epoch [7/100], Step [61/93], Loss: 1.2014
Epoch [7/100], Step [62/93], Loss: 1.7800
Epoch [7/100], Step [63/93], Loss: 0.8145
Epoch [7/100], Step [64/93], Loss: 1.5697
Epoch [7/100], Step [65/93], Loss: 1.0928
Epoch [7/100], Step [66/93], Loss: 1.2865
Epoch [7/100], Step [67/93], Loss: 1.1812
Epoch [7/100], Step [68/93], Loss: 1.5931
Epoch [7/100], Step [69/93], Loss: 1.1221
Epoch [7/100], Step [70/93], Loss: 1.5513
Epoch [7/100], Step [71/93], Loss: 0.7700
Epoch [7/100], Step [72/93], Loss: 1.4254
Epoch [7/100], Step [73/93], Loss: 1.5954
Epoch [7/100], Step [74/93], Loss: 1.2518
Epoch [7/100], Step [75/93], Loss: 0.7342
Epoch [7/100], Step [76/93], Loss: 0.9816
Epoch [7/100], Step [77/93], Loss: 1.4079
Epoch [7/100], Step [78/93], Loss: 1.2896
Epoch [7/100], Step [79/93], Loss: 1.4340
Epoch [7/100], Step [80/93], Loss: 1.3583
Epoch [7/100], Step [81/93], Loss: 1.4695
Epoch [7/100], Step [82/93], Loss:

Epoch [9/100], Step [71/93], Loss: 0.3666
Epoch [9/100], Step [72/93], Loss: 1.4341
Epoch [9/100], Step [73/93], Loss: 1.5965
Epoch [9/100], Step [74/93], Loss: 1.4081
Epoch [9/100], Step [75/93], Loss: 0.3943
Epoch [9/100], Step [76/93], Loss: 0.4648
Epoch [9/100], Step [77/93], Loss: 1.4307
Epoch [9/100], Step [78/93], Loss: 1.3295
Epoch [9/100], Step [79/93], Loss: 1.4445
Epoch [9/100], Step [80/93], Loss: 1.3629
Epoch [9/100], Step [81/93], Loss: 1.4573
Epoch [9/100], Step [82/93], Loss: 1.4823
Epoch [9/100], Step [83/93], Loss: 0.3087
Epoch [9/100], Step [84/93], Loss: 1.4993
Epoch [9/100], Step [85/93], Loss: 1.2062
Epoch [9/100], Step [86/93], Loss: 1.6014
Epoch [9/100], Step [87/93], Loss: 0.7517
Epoch [9/100], Step [88/93], Loss: 1.7321
Epoch [9/100], Step [89/93], Loss: 1.1800
Epoch [9/100], Step [90/93], Loss: 1.4461
Epoch [9/100], Step [91/93], Loss: 0.2491
Epoch [9/100], Step [92/93], Loss: 1.4474
Epoch [9/100], Step [93/93], Loss: 0.6261
Epoch [10/100], Step [1/93], Loss:

...

Epoch [89/100], Step [66/93], Loss: 0.0037
Epoch [89/100], Step [67/93], Loss: 0.0012
Epoch [89/100], Step [68/93], Loss: 0.0062
Epoch [89/100], Step [69/93], Loss: 0.0002
Epoch [89/100], Step [70/93], Loss: 0.0018
Epoch [89/100], Step [71/93], Loss: 0.0000
Epoch [89/100], Step [72/93], Loss: 0.0004
Epoch [89/100], Step [73/93], Loss: 0.0019
Epoch [89/100], Step [74/93], Loss: 0.0000
Epoch [89/100], Step [75/93], Loss: 0.0000
Epoch [89/100], Step [76/93], Loss: 0.0083
Epoch [89/100], Step [77/93], Loss: 0.0069
Epoch [89/100], Step [78/93], Loss: -0.0000
Epoch [89/100], Step [79/93], Loss: 0.0084
Epoch [89/100], Step [80/93], Loss: 0.0016
Epoch [89/100], Step [81/93], Loss: 0.0012
Epoch [89/100], Step [82/93], Loss: 0.0002
Epoch [89/100], Step [83/93], Loss: 0.0000
Epoch [89/100], Step [84/93], Loss: 0.0003
Epoch [89/100], Step [85/93], Loss: 0.0003
Epoch [89/100], Step [86/93], Loss: 0.0000
Epoch [89/100], Step [87/93], Loss: 0.0338
Epoch [89/100], Step [88/93], Loss: 0.0048
Epoch [89/

Epoch [92/100], Step [7/93], Loss: 0.0123
Epoch [92/100], Step [8/93], Loss: 0.0066
Epoch [92/100], Step [9/93], Loss: 0.0031
Epoch [92/100], Step [10/93], Loss: 0.0010
Epoch [92/100], Step [11/93], Loss: 0.0030
Epoch [92/100], Step [12/93], Loss: 0.0002
Epoch [92/100], Step [13/93], Loss: -0.0000
Epoch [92/100], Step [14/93], Loss: -0.0000
Epoch [92/100], Step [15/93], Loss: 0.0000
Epoch [92/100], Step [16/93], Loss: 0.0040
Epoch [92/100], Step [17/93], Loss: 0.0005
Epoch [92/100], Step [18/93], Loss: 0.0016
Epoch [92/100], Step [19/93], Loss: 0.0002
Epoch [92/100], Step [20/93], Loss: 0.0000
Epoch [92/100], Step [21/93], Loss: 0.0004
Epoch [92/100], Step [22/93], Loss: 0.0000
Epoch [92/100], Step [23/93], Loss: 0.0023
Epoch [92/100], Step [24/93], Loss: 0.0002
Epoch [92/100], Step [25/93], Loss: 0.0006
Epoch [92/100], Step [26/93], Loss: 0.0012
Epoch [92/100], Step [27/93], Loss: 0.0087
Epoch [92/100], Step [28/93], Loss: 0.0002
Epoch [92/100], Step [29/93], Loss: 0.0000
Epoch [92/10

Epoch [94/100], Step [44/93], Loss: 1.0063
Epoch [94/100], Step [45/93], Loss: 0.0000
Epoch [94/100], Step [46/93], Loss: 0.0018
Epoch [94/100], Step [47/93], Loss: -0.0000
Epoch [94/100], Step [48/93], Loss: 0.0043
Epoch [94/100], Step [49/93], Loss: 0.0012
Epoch [94/100], Step [50/93], Loss: 0.0126
Epoch [94/100], Step [51/93], Loss: 0.0003
Epoch [94/100], Step [52/93], Loss: 0.0000
Epoch [94/100], Step [53/93], Loss: 0.0008
Epoch [94/100], Step [54/93], Loss: 0.0000
Epoch [94/100], Step [55/93], Loss: 0.0091
Epoch [94/100], Step [56/93], Loss: 0.0001
Epoch [94/100], Step [57/93], Loss: 0.0014
Epoch [94/100], Step [58/93], Loss: 0.0055
Epoch [94/100], Step [59/93], Loss: 0.0000
Epoch [94/100], Step [60/93], Loss: 0.0086
Epoch [94/100], Step [61/93], Loss: 0.0028
Epoch [94/100], Step [62/93], Loss: 0.0009
Epoch [94/100], Step [63/93], Loss: 0.0001
Epoch [94/100], Step [64/93], Loss: 0.0003
Epoch [94/100], Step [65/93], Loss: 0.0005
Epoch [94/100], Step [66/93], Loss: 0.0037
Epoch [94/

Epoch [96/100], Step [74/93], Loss: 0.0000
Epoch [96/100], Step [75/93], Loss: 0.0000
Epoch [96/100], Step [76/93], Loss: 0.0073
Epoch [96/100], Step [77/93], Loss: 0.0096
Epoch [96/100], Step [78/93], Loss: -0.0000
Epoch [96/100], Step [79/93], Loss: 0.0063
Epoch [96/100], Step [80/93], Loss: 0.0007
Epoch [96/100], Step [81/93], Loss: 0.0013
Epoch [96/100], Step [82/93], Loss: 0.0004
Epoch [96/100], Step [83/93], Loss: 0.0000
Epoch [96/100], Step [84/93], Loss: 0.0004
Epoch [96/100], Step [85/93], Loss: 0.0005
Epoch [96/100], Step [86/93], Loss: 0.0000
Epoch [96/100], Step [87/93], Loss: 0.0376
Epoch [96/100], Step [88/93], Loss: 0.0028
Epoch [96/100], Step [89/93], Loss: -0.0000
Epoch [96/100], Step [90/93], Loss: 0.0001
Epoch [96/100], Step [91/93], Loss: 0.0008
Epoch [96/100], Step [92/93], Loss: 0.0005
Epoch [96/100], Step [93/93], Loss: 0.0000
Epoch [97/100], Step [1/93], Loss: 0.0058
Epoch [97/100], Step [2/93], Loss: 0.0005
Epoch [97/100], Step [3/93], Loss: 0.0003
Epoch [97/10

Epoch [99/100], Step [19/93], Loss: 0.0003
Epoch [99/100], Step [20/93], Loss: -0.0000
Epoch [99/100], Step [21/93], Loss: 0.0002
Epoch [99/100], Step [22/93], Loss: 0.0000
Epoch [99/100], Step [23/93], Loss: 0.0026
Epoch [99/100], Step [24/93], Loss: 0.0001
Epoch [99/100], Step [25/93], Loss: 0.0003
Epoch [99/100], Step [26/93], Loss: 0.0010
Epoch [99/100], Step [27/93], Loss: 0.0041
Epoch [99/100], Step [28/93], Loss: 0.0002
Epoch [99/100], Step [29/93], Loss: 0.0000
Epoch [99/100], Step [30/93], Loss: 0.0009
Epoch [99/100], Step [31/93], Loss: 0.0001
Epoch [99/100], Step [32/93], Loss: 0.0000
Epoch [99/100], Step [33/93], Loss: -0.0000
Epoch [99/100], Step [34/93], Loss: 0.0000
Epoch [99/100], Step [35/93], Loss: 0.0003
Epoch [99/100], Step [36/93], Loss: 0.0039
Epoch [99/100], Step [37/93], Loss: 0.0014
Epoch [99/100], Step [38/93], Loss: 0.9802
Epoch [99/100], Step [39/93], Loss: 0.0031
Epoch [99/100], Step [40/93], Loss: 0.0009
Epoch [99/100], Step [41/93], Loss: 0.0031
Epoch [99

In [21]:
cx_test.shape


(41, 11, 13)

# Main

In [27]:
acc,cm,rec,prec = [],[],[],[]

a,b,c,d = gaussian(x_train,y_train,x_test,y_test)
acc.append(a)
cm.append(b)
rec.append(c)
prec.append(d)

a,b,c,d = svm(x_train,y_train,x_test,y_test)
acc.append(a)
cm.append(b)
rec.append(c)
prec.append(d)

a,b,c,d = mlp(x_test)
acc.append(a)
cm.append(b)
rec.append(c)
prec.append(d)

a,b,c,d = cnn(cx_test)
acc.append(a)
cm.append(b)
rec.append(c)
prec.append(d)


['kobayashi'] kobayashi


# Print Result

In [28]:
from prettytable import PrettyTable as pt
t=pt(['Methods','Accuracy','Recall','Precision','Confusion_Matrix'])

methods = ["Gaussian NB","SVM","MLP","CNN"]
for i,a,b,c,d in zip(methods,acc,rec,prec,cm):
    t.add_row([i,"{:.6f}".format(a),"{:.6f}".format(b),
           "{:.6f}".format(c),d])
t.align['Methods']='l'
print(t) 

+-------------+----------+----------+-----------+------------------+
| Methods     | Accuracy |  Recall  | Precision | Confusion_Matrix |
+-------------+----------+----------+-----------+------------------+
| Gaussian NB | 0.414634 | 0.409596 |  0.625000 |    [[8 0 0 2]    |
|             |          |          |           |     [5 3 0 3]    |
|             |          |          |           |     [6 0 1 2]    |
|             |          |          |           |    [5 1 0 5]]    |
| SVM         | 0.390244 | 0.390152 |  0.426515 |    [[5 4 0 1]    |
|             |          |          |           |     [0 5 1 5]    |
|             |          |          |           |     [2 2 3 2]    |
|             |          |          |           |    [3 4 1 3]]    |
| MLP         | 0.414634 | 0.413384 |  0.456589 |    [[3 5 2 0]    |
|             |          |          |           |     [0 8 2 1]    |
|             |          |          |           |     [4 1 4 0]    |
|             |          |        